In [1]:
import xmltodict
import os

In [5]:
def ler_xml_danfe(nota):
    with open(f'{nota}', 'rb') as arquivo:
        documento = xmltodict.parse(arquivo)
    dic_notafiscal = documento['nfeProc']['NFe']['infNFe']
    valor_total = dic_notafiscal['total']['ICMSTot']['vNF']
    cnpj = dic_notafiscal['emit']['CNPJ']
    nome_do_vendedor = dic_notafiscal['emit']['xNome']
    cpfcnpj_doComprador = dic_notafiscal['dest']['CPF']
    Nome_doComprador = dic_notafiscal['dest']['xNome']
    produtos = dic_notafiscal['det']
    lista_produtos = []
    for produto in produtos:
        valor_produto = produto['prod']['vProd']
        nome_produto = produto['prod']['xProd']
        lista_produtos.append((nome_produto, valor_produto))
    resultado = {
        'valor_total': valor_total,
        'cnpj_vendeu': cnpj,
        'nome_do_vendedor': nome_do_vendedor,
        'nome_do_comprador': Nome_doComprador,
        'cpf_do_comprador': cpfcnpj_doComprador,
        'lista_produtos': lista_produtos,
    }
    return resultado

def ler_xml_servico(nota):
    with open(f'{nota}', 'rb') as arquivo:
        documento = xmltodict.parse(arquivo)
    dic_notafiscal = documento['ConsultarNfseResposta']['ListaNfse']['CompNfse']['Nfse']['InfNfse']
    valor_total = dic_notafiscal['Servico']['Valores']['ValorServicos']
    cnpj = dic_notafiscal['PrestadorServico']['IdentificacaoPrestador']['Cnpj']
    nome_do_prestador = dic_notafiscal['PrestadorServico']['RazaoSocial']
    cpfcnpj_doComprador = dic_notafiscal['TomadorServico']['IdentificacaoTomador']['CpfCnpj']['Cnpj']
    Nome_doComprador = dic_notafiscal['TomadorServico']['RazaoSocial']
    servicos = dic_notafiscal['Servico']['Discriminacao']

    resultado = {
        'valor_total': valor_total,
        'cnpj_vendeu': cnpj,
        'nome_do_vendedor': nome_do_prestador,
        'nome_do_comprador': Nome_doComprador,
        'cpf_do_comprador': cpfcnpj_doComprador,
        'lista_produtos': servicos,
    }
    return resultado
            

In [3]:
import pandas as pd
import numpy as np  
import openpyxl as opx

In [6]:
arquivos = os.listdir('C:/Users/phsau/Vscode Files/Python/Dados/NFs Finais')

lista_de_resultados = []
for arquivo in arquivos:
    if 'xml' in arquivo:
        if 'DANFE' in arquivo:
            resultado = ler_xml_danfe('C:/Users/phsau/Vscode Files/Python/Dados/NFs Finais/{}'.format(arquivo))
            lista_de_resultados.append(resultado)
        else:
            resultado = ler_xml_servico('C:/Users/phsau/Vscode Files/Python/Dados/NFs Finais/{}'.format(arquivo))
            lista_de_resultados.append(resultado)

tabela = pd.DataFrame(lista_de_resultados)
tabela.to_excel('NFS.xlsx', index=False)
